In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA


In [2]:
data = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

In [ ]:
# a. Data preparation

In [3]:
sentence = data.split('.')

In [4]:
sentence

['We are about to study the idea of a computational process',
 '\nComputational processes are abstract beings that inhabit computers',
 '\nAs they evolve, processes manipulate other abstract things called data',
 '\nThe evolution of a process is directed by a pattern of rules\ncalled a program',
 ' People create programs to direct processes',
 ' In effect,\nwe conjure the spirits of the computer with our spells',
 '']

In [7]:
def is_special_character(char):
    return not char.isalnum() and not char.isspace()

filtered_text = []
for line in sentence:
    words = line.split()
    filtered_words = []
    for word in words:
        # Remove special characters, numbers, and single-character words
        if all((not is_special_character(word), not word.isdigit(), len(word) > 1)):
            filtered_words.append(word)
    filtered_text.append(" ".join(filtered_words))

filtered_text = [line for line in filtered_text if line]  # Remove empty lines
print(filtered_text)

['We are about to study the idea of computational process', 'Computational processes are abstract beings that inhabit computers', 'As they processes manipulate other abstract things called data', 'The evolution of process is directed by pattern of rules called program', 'People create programs to direct processes', 'In we conjure the spirits of the computer with our spells']


In [8]:
data = filtered_text

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)
sequences = tokenizer.texts_to_sequences(data)

print(sequences)

[[4, 5, 11, 6, 12, 1, 13, 2, 7, 8], [7, 3, 5, 9, 14, 15, 16, 17], [18, 19, 3, 20, 21, 9, 22, 10, 23], [1, 24, 2, 8, 25, 26, 27, 28, 2, 29, 10, 30], [31, 32, 33, 6, 34, 3], [35, 4, 36, 1, 37, 2, 1, 38, 39, 40, 41]]


In [14]:

# Initialize dictionaries for word to ID and ID to word mappings
word2id = {}
id2word = {}

# Iterate through each sentence and assign IDs to words
current_id = 0
for sentence in filtered_text:
    words = sentence.split()
    for word in words:
        if word not in word2id:
            word2id[word] = current_id
            id2word[current_id] = word
            current_id += 1

# Now you can use word2id to get the ID of a word and id2word to get the word from its ID
print("Word to ID mapping:")
print(word2id)

print("ID to Word mapping:")
print(id2word)


Word to ID mapping:
{'We': 0, 'are': 1, 'about': 2, 'to': 3, 'study': 4, 'the': 5, 'idea': 6, 'of': 7, 'computational': 8, 'process': 9, 'Computational': 10, 'processes': 11, 'abstract': 12, 'beings': 13, 'that': 14, 'inhabit': 15, 'computers': 16, 'As': 17, 'they': 18, 'manipulate': 19, 'other': 20, 'things': 21, 'called': 22, 'data': 23, 'The': 24, 'evolution': 25, 'is': 26, 'directed': 27, 'by': 28, 'pattern': 29, 'rules': 30, 'program': 31, 'People': 32, 'create': 33, 'programs': 34, 'direct': 35, 'In': 36, 'we': 37, 'conjure': 38, 'spirits': 39, 'computer': 40, 'with': 41, 'our': 42, 'spells': 43}
ID to Word mapping:
{0: 'We', 1: 'are', 2: 'about', 3: 'to', 4: 'study', 5: 'the', 6: 'idea', 7: 'of', 8: 'computational', 9: 'process', 10: 'Computational', 11: 'processes', 12: 'abstract', 13: 'beings', 14: 'that', 15: 'inhabit', 16: 'computers', 17: 'As', 18: 'they', 19: 'manipulate', 20: 'other', 21: 'things', 22: 'called', 23: 'data', 24: 'The', 25: 'evolution', 26: 'is', 27: 'direc

In [20]:
vocab_size = len(tokenizer.word_index) + 1
embedding_size = 10
window_size = 2

# Generate the context-target pairs
contexts = []
targets = []
for sequence in sequences:
	for i in range(window_size, len(sequence) - window_size):
		context = sequence[i - window_size:i] + sequence[i + 1:i + window_size + 1]
		target = sequence[i]
		contexts.append(context)
		targets.append(target)

In [22]:
# sample of training data
for i in range(5):
    words = []
    target = id2word.get(targets[i])
    for j in contexts[i]:
        words.append(id2word.get(j))
    print(words, "=>", target)

['study', 'the', 'idea', 'abstract'] => processes
['the', 'processes', 'abstract', 'are'] => idea
['processes', 'idea', 'are', 'beings'] => abstract
['idea', 'abstract', 'beings', 'about'] => are
['abstract', 'are', 'about', 'of'] => beings


In [23]:
# Convert the contexts and targets to numpy arrays
X = np.array(contexts)
Y = np.array(targets)

In [25]:
# Define the CBOW model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,\
	Embedding, Lambda

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=2 * window_size))
model.add(Lambda(lambda x: tf.reduce_mean(x, axis=1)))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(units=vocab_size, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X, Y, epochs=200, verbose=1)

Epoch 1/200
1/1 [==============================] - 1s 761ms/step - loss: 3.7372 - accuracy: 0.0625
Epoch 2/200
1/1 [==============================] - 0s 967us/step - loss: 3.7309 - accuracy: 0.1250
Epoch 3/200
1/1 [==============================] - 0s 19ms/step - loss: 3.7246 - accuracy: 0.1562
Epoch 4/200
1/1 [==============================] - 0s 18ms/step - loss: 3.7176 - accuracy: 0.1250
Epoch 5/200
1/1 [==============================] - 0s 5ms/step - loss: 3.7096 - accuracy: 0.1250
Epoch 6/200
1/1 [==============================] - 0s 15ms/step - loss: 3.7003 - accuracy: 0.1250
Epoch 7/200
1/1 [==============================] - 0s 15ms/step - loss: 3.6895 - accuracy: 0.1250
Epoch 8/200
1/1 [==============================] - 0s 16ms/step - loss: 3.6770 - accuracy: 0.1250
Epoch 9/200
1/1 [==============================] - 0s 10ms/step - loss: 3.6624 - accuracy: 0.1250
Epoch 10/200
1/1 [==============================] - 0s 8ms/step - loss: 3.6456 - accuracy: 0.1250
Epoch 11/200
1/1 [=

1/1 [==============================] - 0s 7ms/step - loss: 0.3857 - accuracy: 1.0000
Epoch 85/200
1/1 [==============================] - 0s 12ms/step - loss: 0.3594 - accuracy: 1.0000
Epoch 86/200
1/1 [==============================] - 0s 2ms/step - loss: 0.3347 - accuracy: 1.0000
Epoch 87/200
1/1 [==============================] - 0s 15ms/step - loss: 0.3115 - accuracy: 1.0000
Epoch 88/200
1/1 [==============================] - 0s 0s/step - loss: 0.2898 - accuracy: 1.0000
Epoch 89/200
1/1 [==============================] - 0s 5ms/step - loss: 0.2696 - accuracy: 1.0000
Epoch 90/200
1/1 [==============================] - 0s 14ms/step - loss: 0.2508 - accuracy: 1.0000
Epoch 91/200
1/1 [==============================] - 0s 0s/step - loss: 0.2333 - accuracy: 1.0000
Epoch 92/200
1/1 [==============================] - 0s 17ms/step - loss: 0.2169 - accuracy: 1.0000
Epoch 93/200
1/1 [==============================] - 0s 14ms/step - loss: 0.2018 - accuracy: 1.0000
Epoch 94/200
1/1 [============

1/1 [==============================] - 0s 0s/step - loss: 0.0083 - accuracy: 1.0000
Epoch 167/200
1/1 [==============================] - 0s 19ms/step - loss: 0.0082 - accuracy: 1.0000
Epoch 168/200
1/1 [==============================] - 0s 10ms/step - loss: 0.0080 - accuracy: 1.0000
Epoch 169/200
1/1 [==============================] - 0s 2ms/step - loss: 0.0079 - accuracy: 1.0000
Epoch 170/200
1/1 [==============================] - 0s 17ms/step - loss: 0.0078 - accuracy: 1.0000
Epoch 171/200
1/1 [==============================] - 0s 0s/step - loss: 0.0076 - accuracy: 1.0000
Epoch 172/200
1/1 [==============================] - 0s 3ms/step - loss: 0.0075 - accuracy: 1.0000
Epoch 173/200
1/1 [==============================] - 0s 14ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 174/200
1/1 [==============================] - 0s 0s/step - loss: 0.0073 - accuracy: 1.0000
Epoch 175/200
1/1 [==============================] - 0s 18ms/step - loss: 0.0071 - accuracy: 1.0000
Epoch 176/200
1/1 [===

In [26]:
# test model
test_sentenses = [
    "we are to study",
    "create programs direct processes",
    "spirits process study program",
    "idea study people create"
]

In [40]:
# Define the test sentence
test_sentence = "we are to study"

# Split the sentence into words
test_words = test_sentence.split(" ")

# Map words to their IDs
x_test = np.array([word2id.get(word, -1) for word in test_words])



test_predictions = model.predict(np.array([valid_ids]))
y_pred = np.argmax(test_predictions[0])


target_word = id2word[y_pred]
print(f" {valid_words} => {target_word}")


1/1 [==============================] - 0s 20ms/step
 ['we', 'are', 'to', 'study'] => In
